# 头条新闻分类Bert Baseline

## BERT要点
#### BERT why self-attention
+ 计算复杂度，self-attention每层的复杂度O(n^2*d) n是句子的长度，d是词向量维度
![](./table1.png)
+ 可以并行
+ 长程依赖，LSTM任意两点之间需要经过一定的距离，Attention任意两点之间可以直接进行计算。

#### 主要贡献
+ BERT使用掩码语言模型，可以使得预训练模型进行双向表示
+ BERT是第一个基于微调的模型

#### Task 1: MASKed LM(遮蔽语言模型)
为了训练双向深度表示，我们按照百分比（15%）随机遮盖一些token，然后仅预测这些别遮盖的词。
被掩盖的词中，
1. 80%的词 被替换成 [MASK]
2. 10%的词 被随机替换
3. 10%的词 不动

#### Task2：Next Sentence Prediction 
(A, B) 其中B有50%的概率是A的下一句，50%的概率是从数据集中随机选择的一句。
如果B是A的下一句标注成isNexT，不是则被标注成NotNext。

## 编写配置

In [1]:
import torch 
import torch.nn as nn

config = {
    'train_file_path': '../../../data/toutiao_news_cls/train.csv',
    'test_file_path': '../../../data/toutiao_news_cls/test.csv',
    'train_val_ratio': 0.1,  # 10%用作验证集
    # ------ 与TextCNN不同的配置 ------
    # 'vocab_size': 10000,   # 词典 3W
    'head': 'cnn',
    'model_path': '../../../pt/bert-base-chinese',
    # ------ 与TextCNN不同的配置 ------
    'batch_size': 16,      # batch 大小 16
    'num_epochs': 1,      # 10次迭代
    'learning_rate': 2e-5, # 学习率
    'logging_step': 300,   # 每跑300个batch记录一次
    'seed': 2022           # 随机种子
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu' # cpu&gpu

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

## 数据预处理并编写DataLoader

In [2]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import BertTokenizer
from torch.utils.data import DataLoader

In [3]:
# bert分词器
bertTokenizer = BertTokenizer.from_pretrained(config['model_path'])
# 重写分词器
def tokenizer(sent):
    inputs = bertTokenizer.encode_plus(sent, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True)
    
    return inputs


In [4]:
def read_data(config, mode='train'):
    
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    LABEL, SENTENCE = 'label', 'sentence'
    data_df['bert_encode'] = data_df[SENTENCE].apply(tokenizer)
    data_df['input_ids'] = data_df['bert_encode'].apply(lambda s: s['input_ids'])
    input_ids = np.array([[int(id_) for id_ in v] for v in data_df['input_ids'].values])
    data_df['token_type_ids'] = data_df['bert_encode'].apply(lambda s: s['token_type_ids'])
    token_type_ids = np.array([[int(id_) for id_ in v] for v in data_df['token_type_ids'].values])
    data_df['attention_mask'] = data_df['bert_encode'].apply(lambda s: s['attention_mask'])
    attention_mask = np.array([[int(id_) for id_ in v] for v in data_df['attention_mask'].values])

    if mode == 'train':
        labels = data_df[LABEL].values
        
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(config['train_val_ratio'] * len(data_df))
        
        # shuffle ids
        ids = np.random.choice(range(len(data_df)), size=len(data_df), replace=False)
        train_ids = ids[num_val:]
        val_ids = ids[:num_val]
        
        # get input_ids
        X_train['input_ids'], y_train = input_ids[train_ids], labels[train_ids]
        X_val['input_ids'], y_val = input_ids[val_ids], labels[val_ids]
         # get token_type_ids
        X_train['token_type_ids'] = token_type_ids[train_ids]
        X_val['token_type_ids'] = token_type_ids[val_ids]
        # get attention_mask
        X_train['attention_mask'] = attention_mask[train_ids]
        X_val['attention_mask'] = attention_mask[val_ids]
     
        # label 
        label2id = {label: i for i, label in enumerate(np.unique(y_train))}
        id2label = {i: label for label, i in label2id.items()}
        y_train = torch.tensor([label2id[y] for y in y_train], dtype=torch.long)
        y_val = torch.tensor([label2id[y] for y in y_val], dtype=torch.long)

        return X_train, y_train, X_val, y_val, label2id, id2label

    else:
        X_test = defaultdict(list)
        X_test['input_ids'] = input_ids
        X_test['token_type_ids'] = token_type_ids
        X_test['attention_mask'] = attention_mask
        y_test = torch.zeros(len(data_df), dtype=torch.long)
        
        return X_test, y_test

In [5]:
X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, mode='train')

/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

In [6]:
X_test, y_test = read_data(config, mode='test')

/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

#### Dataset提供数据集的封装，创建/继承Dataset必须实现:
+ __len__: 整个数据集的长度
+ __getitem__: 支持数据集索引的函数

In [7]:
from torch.utils.data import Dataset
class TNEWSDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __getitem__(self, idx):
        return {
            'input_ids' : self.x['input_ids'][idx],
            'label' : self.y[idx],
            'token_type_ids': self.x['token_type_ids'][idx],
            'attention_mask': self.x['attention_mask'][idx]
        }
    
    def __len__(self):
        return self.y.size(0)

#### 使用DataLoader实现数据集的并行加载
+ DataLoader提供一个可迭代对象，实现数据并行加载，从TNEWSDataset返回一个example，取多次，最后形成一个长度为batch_size的列表examples
+ examples的格式：[dict1, dict2, ...]
+ collate_fn()将examples中的数据合并为Tensor

In [8]:
def collate_fn(examples):
    input_ids_lst = []
    labels = []
    # ------ 与TextCNN不同的地方 ------
    token_type_ids_lst = []
    attention_mask_lst = []
    # ------ 与TextCNN不同的地方 ------

    for example in examples:
        input_ids_lst.append(example['input_ids'])
        labels.append(example['label'])
        # ------ 与TextCNN不同的地方 ------
        token_type_ids_lst.append(example['token_type_ids'])
        attention_mask_lst.append(example['attention_mask'])
        # ------ 与TextCNN不同的地方 ------
        
    # 计算input_ids_lst中最长的句子长度，对齐
    max_length = max(len(input_ids) for input_ids in input_ids_lst)
    # 定义一个Tensor
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)
    # ------ 与TextCNN不同的地方 ------
    token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
    attention_mask_tensor = torch.zeros_like(input_ids_tensor)
    # ------ 与TextCNN不同的地方 ------
    
    for i, input_ids in enumerate(input_ids_lst):
        seq_len = len(input_ids)
        input_ids_tensor[i, :seq_len] = torch.tensor(input_ids, dtype=torch.long)
        # ------ 与TextCNN不同的地方 ------
        token_type_ids_tensor[i, :seq_len] = torch.tensor(token_type_ids_lst[i], dtype=torch.long)
        attention_mask_tensor[i, :seq_len] = torch.tensor(attention_mask_lst[i], dtype=torch.long)
        # ------ 与TextCNN不同的地方 ------
        
    return {
        'input_ids': input_ids_tensor,
        'labels': torch.tensor(labels, dtype=torch.long),
        # ------ 与TextCNN不同的地方 ------
        'token_type_ids': token_type_ids_tensor,
        'attention_mask': attention_mask_tensor
        # ------ 与TextCNN不同的地方 ------
    }

In [9]:
from torch.utils.data import DataLoader

def build_dataloader(config):
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, mode='train')
    X_test, y_test = read_data(config, mode='test')
    
    train_dataset = TNEWSDataset(X_train, y_train)
    val_dataset = TNEWSDataset(X_val, y_val)
    test_dataset = TNEWSDataset(X_test, y_test)
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=False, collate_fn=collate_fn)

    return train_dataloader, val_dataloader, test_dataloader, id2label

In [10]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

In [11]:
for batch in train_dataloader:
    print(len(batch['input_ids']), len(batch['labels']), len(batch['token_type_ids']), len(batch['attention_mask']))
    print(batch)
    break

16 16 16 16
{'input_ids': tensor([[ 101,  776,  691, 2356,  966, 5892, 1355, 1283,  783, 8024, 2832, 6598,
         5442,  812,  711,  862, 2898, 5330,  976, 4958,  776,  691, 8043,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101,  677, 5468, 8038, 7599, 3235, 1915, 7455, 3449,  771, 7676, 8024,
          678, 5468, 2582,  720, 2190, 8043,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 1266,  776, 1957, 2094, 1745, 7063, 8038, 4385, 2141, 5445, 3655,
         6999, 8024, 6821, 2218, 3221, 4495, 3833, 8013,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101,  100, 2207, 1285, 1159,  100, 3173, 3124, 5862, 1765, 2157, 7270,
         4193, 5991, 6421,  679, 6421, 5314, 2015, 2845,  702, 4408,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2809, 3124,  671, 1453, 2399, 8024, 7716, 1046, 7987, 6

## 训练验证

In [12]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation...', total=len(val_dataloader))
    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch = {item:value.to(config['device']) for item, value in batch.items()}
            
            # val output (loss, out)
            loss, logits = model(**batch)[:2]
            val_loss += loss.item()
            
            preds.append(logits.argmax(dim=-1).detach().cpu())
            
    avg_val_loss = val_loss/len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    f1 =f1_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)
    
    return avg_val_loss, f1, precision, recall, accuracy

In [13]:
# Bert model train
from transformers import BertConfig, BertForSequenceClassification
from transformers import AdamW
from tqdm import trange

def train(config, train_dataloader, val_dataloader, model):

    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])
    
    model.to(config['device'])
    
    epoches_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.
    
    best_f1 = 0.
    best_precision = 0.
    best_recall = 0.
    best_accuracy = 0.
    
    for epoch in epoches_iterator:
        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        for batch in train_iterator:
            batch = {item:value.to(config['device']) for item, value in batch.items()}
            
            # train output (loss, out)
            loss = model(**batch)[0]
            
            model.zero_grad()  # 模型参数梯度清零
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            train_loss += loss.item()  # 叠加loss
            global_steps += 1
            
            if global_steps % config['logging_step'] == 0:
                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss
                avg_val_loss, f1, precision, recall, accuracy = evaluation(config, model, val_dataloader)

                if best_f1 < f1:
                    best_f1 = f1
                    best_precision = precision
                    best_recall = recall
                    best_accuracy = accuracy
                    print_log = f'''>>> training loss: {print_train_loss: .4f}, valid loss: {avg_val_loss: .4f}\n
                            valid f1 score: {f1: .4f}, valid precision score: {precision: .4f},
                            valid recall score: {recall: .4f}, valid accuracy score: {accuracy: .4f}'''
                    print(print_log)
                    model.save_pretrained('../../../pt_tmp/bert_base_chinese')
                    
                model.train()
                
    return best_f1, best_precision, best_recall, best_accuracy

In [15]:
# 首次运行代码
# bert_config = BertConfig.from_pretrained(config['model_path'])
# bert_config.num_labels = len(id2label)
# model = BertForSequenceClassification.from_pretrained(config['model_path'], config=bert_config)
# best_model, f1, precision, recall, accuracy = train(config, train_dataloader, val_dataloader, model)
# best_model.save_pretrained('../../../pt_tmp/bert_base_chinese')
# print_log = f'''valid f1 score: {f1: .4f}, valid precision score: {precision: .4f},
#                 valid recall score: {recall: .4f}, valid accuracy score: {accuracy: .4f}'''
# print(print_log)

# 迭代训练代码
bert_config = BertConfig.from_pretrained('../../../pt_tmp/bert_base_chinese')
bert_config.num_labels = len(id2label)
model = BertForSequenceClassification.from_pretrained('../../../pt_tmp/bert_base_chinese')
f1, precision, recall, accuracy = train(config, train_dataloader, val_dataloader, model)
print_log = f'''valid f1 score: {f1: .4f}, valid precision score: {precision: .4f},
                valid recall score: {recall: .4f}, valid accuracy score: {accuracy: .4f}'''
print(print_log)

/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Training:   5%|██▏                                     | 164/3002 [10:56<3:08:17,  3.98s/it]


Training:  10%|███▉                                    | 299/3002 [19:50<2:44:55,  3.66s/it]

Evaluation...:   0%|                                                | 0/334 [00:00<?, ?it/s]

Evaluation...:   0%|                                        | 1/334 [00:01<06:16,  1.13s/it]

Evaluation...:   1%|▏                                       | 2/334 [00:02<05:37,  1.02s/it]

Evaluation...:   1%|▎                                       | 3/334 [00:02<05:18,  1.04it/s]

Evaluation...:   1%|▍                                       | 4/334 [00:03<05:07,  1.07it/s]

Evaluation...:   1%|▌                                       | 5/334 [00:04<05:12,  1.05it/s]

Evaluation...:   2%|▋                                       | 6/334 [00:05<05:01,  1.09it/s]

Evaluation...:   2%|▊                                       | 7/334 [00:06<04:57,  1.10it/s]

Evaluation...:   2%|▉                                       | 8/334 [00:07<04:54,  1.11it/s]

Evaluation...:   3%|█                                       

Evaluation...:  10%|███▊                                   | 33/334 [00:31<04:57,  1.01it/s]

Evaluation...:  10%|███▉                                   | 34/334 [00:32<05:15,  1.05s/it]

Evaluation...:  10%|████                                   | 35/334 [00:33<05:10,  1.04s/it]

Evaluation...:  11%|████▏                                  | 36/334 [00:34<05:05,  1.03s/it]

Evaluation...:  11%|████▎                                  | 37/334 [00:35<05:14,  1.06s/it]

Evaluation...:  11%|████▍                                  | 38/334 [00:36<04:50,  1.02it/s]

Evaluation...:  12%|████▌                                  | 39/334 [00:37<05:12,  1.06s/it]

Evaluation...:  12%|████▋                                  | 40/334 [00:38<04:56,  1.01s/it]

Evaluation...:  12%|████▊                                  | 41/334 [00:39<04:50,  1.01it/s]

Evaluation...:  13%|████▉                                  | 42/334 [00:40<04:52,  1.00s/it]

Evaluation...:  13%|█████                                  |

Evaluation...:  58%|██████████████████████▏               | 195/334 [03:10<02:10,  1.06it/s]

Evaluation...:  59%|██████████████████████▎               | 196/334 [03:11<02:16,  1.01it/s]

Evaluation...:  59%|██████████████████████▍               | 197/334 [03:12<02:14,  1.02it/s]

Evaluation...:  59%|██████████████████████▌               | 198/334 [03:13<02:12,  1.03it/s]

Evaluation...:  60%|██████████████████████▋               | 199/334 [03:14<02:09,  1.04it/s]

Evaluation...:  60%|██████████████████████▊               | 200/334 [03:15<02:06,  1.06it/s]

Evaluation...:  60%|██████████████████████▊               | 201/334 [03:16<02:03,  1.08it/s]

Evaluation...:  60%|██████████████████████▉               | 202/334 [03:17<02:01,  1.09it/s]

Evaluation...:  61%|███████████████████████               | 203/334 [03:18<02:17,  1.05s/it]

Evaluation...:  61%|███████████████████████▏              | 204/334 [03:19<02:11,  1.01s/it]

Evaluation...:  61%|███████████████████████▎              | 

>>> training loss:  1.0147, valid loss:  1.2348

                            valid f1 score:  0.5281, valid precision score:  0.5349,
                            valid recall score:  0.5284, valid accuracy score:  0.5598


...

Training: 100%|██████████████████████████████████████▉| 2999/3002 [4:09:50<00:10,  3.56s/it]

Evaluation...:   0%|                                                | 0/334 [00:00<?, ?it/s]

Evaluation...:   0%|                                        | 1/334 [00:01<06:24,  1.15s/it]

Evaluation...:   1%|▏                                       | 2/334 [00:02<05:46,  1.04s/it]

Evaluation...:   1%|▎                                       | 3/334 [00:03<05:24,  1.02it/s]

Evaluation...:   1%|▍                                       | 4/334 [00:03<05:13,  1.05it/s]

Evaluation...:   1%|▌                                       | 5/334 [00:04<05:17,  1.04it/s]

Evaluation...:   2%|▋                                       | 6/334 [00:05<05:06,  1.07it/s]

Evaluation...:   2%|▊                                       | 7/334 [00:06<05:02,  1.08it/s]

Evaluation...:   2%|▉                                       | 8/334 [00:07<05:01,  1.08it/s]

Evaluation...:   3%|█                                       

Evaluation...:  37%|█████████████▉                        | 122/334 [02:00<03:45,  1.06s/it]

Evaluation...:  37%|█████████████▉                        | 123/334 [02:01<03:34,  1.02s/it]

Evaluation...:  37%|██████████████                        | 124/334 [02:02<03:35,  1.03s/it]

Evaluation...:  37%|██████████████▏                       | 125/334 [02:03<03:37,  1.04s/it]

Evaluation...:  38%|██████████████▎                       | 126/334 [02:04<03:30,  1.01s/it]

Evaluation...:  38%|██████████████▍                       | 127/334 [02:05<03:23,  1.02it/s]

Evaluation...:  38%|██████████████▌                       | 128/334 [02:06<03:17,  1.05it/s]

Evaluation...:  39%|██████████████▋                       | 129/334 [02:07<03:14,  1.05it/s]

Evaluation...:  39%|██████████████▊                       | 130/334 [02:07<03:11,  1.07it/s]

Evaluation...:  39%|██████████████▉                       | 131/334 [02:09<03:16,  1.03it/s]

Evaluation...:  40%|███████████████                       | 

Evaluation...:  85%|████████████████████████████████▎     | 284/334 [30:44<03:40,  4.41s/it]

Evaluation...:  85%|████████████████████████████████▍     | 285/334 [30:48<03:33,  4.35s/it]

Evaluation...:  86%|████████████████████████████████▌     | 286/334 [30:52<03:24,  4.27s/it]

Evaluation...:  86%|████████████████████████████████▋     | 287/334 [30:57<03:24,  4.36s/it]

Evaluation...:  86%|███████████████████████████████     | 288/334 [35:44<1:08:15, 89.03s/it]

Evaluation...:  87%|████████████████████████████████▉     | 289/334 [35:44<46:55, 62.57s/it]

Evaluation...:  87%|████████████████████████████████▉     | 290/334 [35:45<32:20, 44.11s/it]

Evaluation...:  87%|█████████████████████████████████     | 291/334 [35:46<22:20, 31.17s/it]

Evaluation...:  87%|█████████████████████████████████▏    | 292/334 [35:47<15:27, 22.09s/it]

Evaluation...:  88%|█████████████████████████████████▎    | 293/334 [35:48<10:45, 15.73s/it]

Evaluation...:  88%|█████████████████████████████████▍    | 

>>> training loss:  1.0156, valid loss:  1.2002

                            valid f1 score:  0.5709, valid precision score:  0.5698,
                            valid recall score:  0.5762, valid accuracy score:  0.5690



100%|████████████████████████████████████████████████████| 1/1 [4:57:08<00:00, 17828.56s/it]


## 预测并保存结果

In [16]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Testing', total=len(test_dataloader))
    model.eval()
    test_preds = []
    
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}

            logits = model(**batch)[1]
            test_preds.append(logits.argmax(dim=-1).detach().cpu())
            
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]
        
    test_df = pd.read_csv(config['test_file_path'], sep=',')
    # test_df.insert(1, column=['label_pred'], value=test_preds)
    test_df['label_pred'] = test_preds
    # test_df.drop(columns=['sentence'], inplace=True)
    test_df.to_csv('submission.csv', index=False, encoding='utf8')

In [17]:
predict(config, id2label, best_model, test_dataloader)

Testing: 100%|████████████████████████████████████████████| 625/625 [10:52<00:00,  1.04s/it]


In [18]:
test_df = pd.read_csv(config['test_file_path'], sep=',')

In [19]:
train_df = pd.read_csv(config['train_file_path'], sep=',')

In [20]:
train_df.head(10)

,id,label,label_desc,sentence
0,0,108,news_edu,上课时学生手机响个不停，老师一怒之下把手机摔了，家长拿发票让老师赔，大家怎么看待这种事？
1,1,104,news_finance,商赢环球股份有限公司关于延期回复上海证券交易所对公司2017年年度报告的事后审核问询函的公告
2,2,106,news_house,通过中介公司买了二手房，首付都付了，现在卖家不想卖了。怎么处理？
3,3,112,news_travel,2018年去俄罗斯看世界杯得花多少钱？
4,4,109,news_tech,剃须刀的个性革新，雷明登天猫定制版新品首发
5,5,103,news_sports,再次证明了“无敌是多么寂寞”——逆天的中国乒乓球队！
6,6,109,news_tech,三农盾SACC-全球首个推出：互联网+区块链+农产品的电商平台
7,7,116,news_game,重做or新英雄？其实重做对暴雪来说同样重要
8,8,103,news_sports,如何在商业活动中不受人欺骗？
9,9,101,news_culture,87版红楼梦最温柔的四个丫鬟，娶谁都是一生的福气


In [39]:
train_df['label'].unique()

array([108, 104, 106, 112, 109, 103, 116, 101, 107, 100, 102, 110, 115,
       113, 114])